1. tf.matmul 메소드를 이용해서 중첩된 binary classification을 구현하고,  
2. tf.nn.softmax 메소드를 이용해서 가능성을 계산한다. 이것이 hypothesis 최종 함수임.  
3. cross entropy 함수의 정의를 이용해서 cost 함수를 정의한다.  

[hunkim/DeepLearningZeroToAll](https://github.com/hunkim/DeepLearningZeroToAll/blob/master/tf2/tf2-06-1-softmax_classifier.py)  
TensorFlow 2 버전 코드

In [1]:
  
# Lab 6 Softmax Classifier
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility


x_data = [[1, 2, 1, 1], #입력 데이터셋은 큰 의미는 보이지 않는 숫자조합이 주어졌음
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1], #3번째 인자가 정답일 경우
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0], #2번째 인자가 정답일 경우
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0], #1번째 인자가 정답일 경우
          [1, 0, 0]]

In [2]:
X = tf.placeholder("float", [None, 4]) 
Y = tf.placeholder("float", [None, 3]) #원핫인코딩을 위해 정답의 개수가 y의 개수임.
nb_classes = 3 #종류가 3개

In [3]:
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight') #입력이 4개이므로 4
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
#행렬식 모델을 생각하면 편함

In [6]:
#softmax를 이용한 가설함수, cost함수 구현. softmax함수 안에 들어가는 인자가 logit임.
#즉, logit -> softmax -> probability

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [11]:
#학습, 과정 출력(cost_val이 감소하는것을 확인 가능)
#여기서는 x의 숫자조합이 큰 의미가 없으니 과정 자체만 확인.
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
            _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})

            if step % 200 == 0:
                print(step, cost_val)

    print('--------------')
    # Testing & One-hot encoding  
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1))) 
    #a를 의미하는 예측값과 그것의 인코딩값(몇 번째 인덱스가 정답인지 알려주는 값) 출력

    print('--------------')
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))
    #잘 보면 가장 큰 값(특히 소수점 자리수가 가장 작은)이 몇 번째에 있는지 확인 가능


0 6.926112
200 0.6005018
400 0.47295803
600 0.37342983
800 0.28018403
1000 0.23280531
1200 0.21065348
1400 0.19229904
1600 0.17682338
1800 0.16359547
2000 0.15216155
--------------
[[1.3890503e-03 9.9860185e-01 9.0612566e-06]] [1]
--------------
[[0.9311919  0.06290218 0.00590591]] [0]
--------------
[[1.2732840e-08 3.3411290e-04 9.9966586e-01]] [2]
--------------
[[1.3890502e-03 9.9860185e-01 9.0612657e-06]
 [9.3119192e-01 6.2902153e-02 5.9059118e-03]
 [1.2732840e-08 3.3411290e-04 9.9966586e-01]] [1 0 2]
